In [1]:
from mysql.connector import MySQLConnection, Error
import pandas as pd
from sqlalchemy import create_engine 
import pymysql

from connection_config import read_db_config

In [2]:
conn_string,ssl_args = read_db_config('teama.teama')
engine = create_engine(conn_string,connect_args=ssl_args)

## Loading total load tables
Tables with raw data:
- totalload_2006_2015_entsoe_raw
- totalload_2016_entsoe_raw
- totalload_2017_entsoe_raw
- totalload_2018_entsoe_raw
- totalload_2019_entsoe_raw
- totalload_2020_entsoe_raw

In [3]:
# Read total_load_2020
total_load_2020= pd.read_sql('SELECT * FROM totalload_2020_entsoe_raw', con=engine)

In [4]:
total_load_2020.head()

,index,Time (CET),Day-ahead Total Load Forecast [MW] - United Kingdom (UK),Actual Total Load [MW] - United Kingdom (UK)
0,0,01.01.2020 00:00 - 01.01.2020 00:30,28407,31659
1,1,01.01.2020 00:30 - 01.01.2020 01:00,27994,31331
2,2,01.01.2020 01:00 - 01.01.2020 01:30,28409,31493
3,3,01.01.2020 01:30 - 01.01.2020 02:00,28960,32048
4,4,01.01.2020 02:00 - 01.01.2020 02:30,28347,31712


In [5]:
# Read total_load_2006_20015: as it has information on all countries, I only select GB
total_load_2006_2015= pd.read_sql('SELECT * FROM totalload_2006_2015_entsoe_raw WHERE Country = "GB" ', con=engine)

### Important points regarding the 2006 - 2015 table: 
- the coverage ratio has to be 100 for all instances, scale corresponding hourly values if not 100 (eg. if ratio is 81, the hourly values are = value*1.19)
- sum all 24 (0-23) columns which correspond to hours of the day to have one value per day
- if a few values per hour per day are missing, fill with average of neigbouring values

In [6]:
total_load_2006_2015.tail(5)

,index,Country,Year,Month,Day,Coverage ratio,0,1,2,3,...,14,15,16,17,18,19,20,21,22,23
2186,46009,GB,2015,12,27,81.0,23694.0,23059.0,22312.0,21142.0,...,32886.0,32899.0,33552.0,36575.0,37597.0,36538.0,34759.0,32777.0,30478.0,27981.0
2187,46010,GB,2015,12,28,81.0,25222.0,24173.0,23237.0,21726.0,...,31193.0,31470.0,32473.0,36392.0,38366.0,37198.0,35137.0,32934.0,30383.0,27391.0
2188,46011,GB,2015,12,29,81.0,24338.0,23325.0,22371.0,21075.0,...,31569.0,32193.0,33518.0,37675.0,39932.0,38678.0,36362.0,33847.0,31013.0,27866.0
2189,46012,GB,2015,12,30,81.0,24739.0,23421.0,22485.0,21120.0,...,34978.0,34992.0,35818.0,39070.0,41097.0,40506.0,39120.0,36608.0,33594.0,30426.0
2190,46013,GB,2015,12,31,81.0,27066.0,25890.0,24885.0,23430.0,...,33007.0,33732.0,35022.0,38341.0,39999.0,38667.0,35944.0,33128.0,30488.0,28773.0


### Uploading data to database:
1. Database naming rules: once you clean data, keep the same name and change to raw to clean.
2. Use next cell to upload data onto database
3. If you're not done with cleaning, name it something like intermediate

### Once you upload the data, you can't rewrite it. Running the write command again (trying to create a new table with the same name) will cause error

In [ ]:
#clean_data_dataframe.to_sql('name_of_new_SQL_table',con = engine,if_exists = 'fail')